In [8]:
import pandas as pd
import re
import requests
import difflib
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [11]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get('https://fantasyfootballcalculator.com/adp', headers=header)
ffc = pd.read_html(r.text, header = 0); ffc = ffc[0]

ffc = ffc[['Pick', 'Name', 'Overall', 'Std.Dev', 'High', 'Low']]
ffc = ffc.rename(columns = {'Name': 'name'})


r = requests.get('https://www.fantasypros.com/nfl/adp/overall', headers=header)

fp = pd.read_html(r.text)[0]
fp['Player Team (Bye)'] = fp.iloc[:,1].astype('str')
name = []
for i in range(0,len(fp)):
    try: 
        name.append (re.findall('(.*?)\s[A-Z][A-Z]', fp.iloc[i,1])[0])
    except:
        name.append("")
    

fp['name'] = name
fp = fp.drop('Player Team (Bye)', axis = 1)

stopwords = ["\.", " II", " III", " IV", "Defense", " Jr"]



fp['name'].apply(lambda x: [item for item in x if item not in stopwords])

pat = r'\b(?:{})\b'.format('|'.join(stopwords))
fp['name'] = fp['name'].str.replace(pat, '')
ffc['name'] = ffc['name'].str.replace(pat, '')
fp['name'] = fp['name'].str.replace('.', '')
ffc['name'] = ffc['name'].str.replace('.', '')


fp['name'] = fp['name'].str.upper()
ffc['name'] = ffc['name'].str.upper()

rem_name = []
remainder = ffc[-ffc['name'].isin(fp['name'])]

for i in range(0, len(remainder)):
    rem_name.append( difflib.get_close_matches(remainder.iloc[i,1], fp['name'], cutoff = .55)[0])

indexer = ffc[-ffc['name'].isin(fp['name'])].index
ffc.loc[indexer, "name"] = rem_name



df = ffc.merge(fp)
proj = pd.read_csv('../projections.csv')
proj = proj.dropna()
rem_name = []
for i in range(0, len(df)):
    rem_name.append( difflib.get_close_matches(df.iloc[i,1], proj['name'], cutoff = .15)[0])

df["name"] = rem_name
df = df[['name', 'Overall', 'AVG', 'Std.Dev', 'High', 'Low']]


df['High'] = df['High'] + (12 * (df['High'] - 1.01) ) -.01
df['Low'] = df['Low'] + (12 * (df['Low'] - 1.01) )

df.to_csv('../adp.csv')



In [12]:
df

,name,Overall,AVG,Std.Dev,High,Low
0,SAQUON BARKLEY,1.5,1.0,0.8,1.00,1.40
1,CHRISTIAN MCCAFFREY,2.9,2.5,1.0,1.00,1.66
2,ALVIN KAMARA,3.2,2.5,1.2,1.00,1.79
3,EZEKIEL ELLIOTT,3.7,4.0,1.4,1.00,1.79
4,DEANDRE HOPKINS,5.7,5.3,1.8,1.00,2.31
5,DAVID JOHNSON,6.4,6.5,1.6,1.26,2.44
6,NICK CHUBB,7.3,11.8,1.9,1.26,14.40
7,LE'VEON BELL,7.6,7.8,2.0,1.13,2.44
8,DAVANTE ADAMS,9.2,9.0,2.1,1.39,14.40
9,JAMES CONNER,9.4,8.3,2.3,1.52,14.53
